In [1]:
import os
import sys
import re
import numpy as np
import pickle
from scipy.optimize import fmin
import multiprocessing as mp
import functools

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# statmech library setup
sys.path.append('../../statmechlib')
from statmechlib.preprocessing import select_nodes, to_param_dict, to_param_list, insert_zero_params, rescale_manybody_params
from statmechlib.preprocessing import Trajectory, scale_configuration, pair_dist_cutoff, find_index
from statmechlib.forcefields import sd2_loss, utot_EAM_per_atom, utot_EAM_per_box, ftot_EAM, udif_print, u_core
from statmechlib.preprocessing import universal_eos, downselect
from statmechlib.preprocessing import pair_dist, force_targ, get_stats_EAM_per_atom, get_stats_EAM_per_box
from statmechlib.read_write import params_to_pickle

In [4]:
target_raw = '../data/target_raw'
target_proc = '../data/target_processed'
working = '../data/working'

In [5]:
# stats_file = 'stats_marinica' # 'stats_samples'
# target_file = 'target_marinica' # 'target_samples'
params_file = 'marinica_params'
# stats_file = 'stats_manyknots' # 'stats_samples'
# target_file = 'target_manyknots' # 'target_samples'
# stats_file = 'stats_manymd' # 'stats_samples'
# target_file = 'target_manymd' # 'target_samples'
#stats_file = 'stats_manyall' # 'stats_samples'
#target_file = 'target_manyall' # 'target_samples'
stats_file = 'stats_all_samples' # 'stats_samples'
target_file = 'target_all_samples' # 'target_samples'
# stats_file = 'stats_everything' # 'stats_samples'
# target_file = 'target_everything' # 'target_samples'
stats_md_file = 'stats_onlymd' # 'stats_samples'
target_md_file = 'target_onlymd' # 'stats_samples'

In [6]:
with open(os.path.join(working, stats_file+'.pickle'), 'rb') as fi:
    stats_all = pickle.load(fi)

with open(os.path.join(working, target_file+'.pickle'), 'rb') as fi:
    targets = pickle.load(fi)
    
with open(os.path.join(working, stats_md_file+'.pickle'), 'rb') as fi:
    stats_md = pickle.load(fi)

with open(os.path.join(working, target_md_file+'.pickle'), 'rb') as fi:
    targets_md = pickle.load(fi)

with open(os.path.join(working, params_file + '.pickle'), 'rb') as fi:
    param_list = pickle.load(fi)  

In [7]:
# pair_knots = [2.22, 2.46, 2.5648975,  2.629795 ,  2.6946925,  2.8663175,  2.973045 ,
#         3.0797725,  3.5164725,  3.846445 ,  4.1764175,  4.700845 ,
#         4.8953   ,  5.089755 ,  5.3429525,  5.401695 ,  5.4604375]
# edens_knots = [ 2.22, 2.5,  2.6946925, 2.8663175, 3.1,  3.5,  4.5]

# pair_knots = [2.22, 2.46, 2.5648975, 2.629795, 2.6946925, 2.8663175, 2.973045,
#               3.0797725, 3.5164725, 3.846445, 4.1764175, 4.700845,
#               4.8953, 5.089755, 5.3429525, 5.401695, 5.4604375]

# pair_knots = [2.10, 2.22, 2.46, 2.5648975, 2.629795, 2.6946925, 2.8663175, 2.973045,
#               3.0797725, 3.5164725, 3.846445, 4.1764175, 4.700845,
#               4.8953, 5.089755, 5.3429525,5.401695,5.4604375, 5.54,5.58,5.62]             
#              5.18, 5.38, 5.58, 5.62]


#edens_knots = [2.5, 2.8663175000000001, 3.1, 3.5, 4.5]
#edens_knots = [2.5, 2.8663175000000001, 3.1, 3.5, 4.62]
#edens_knots = [2.22, 2.5, 2.8663175000000001, 3.1, 3.5, 4.62]

pair_knots = [2.5648975, 2.629795, 2.6946925, 2.8663175, 2.973045,
              3.0797725, 3.5164725, 3.846445, 4.1764175, 4.700845,
              4.8953, 5.089755, 5.3429525,5.401695,5.4604375]  
edens_knots = [2.5, 3.1, 3.5, 4.9]



In [8]:
stats = downselect(stats_all, pair_knots, edens_knots)
print(stats['hyperparams'])
stats_select_md = downselect(stats_md, pair_knots, edens_knots)
print(stats_select_md['hyperparams'])

{u'pair': [2.5648974999999998, 2.6297950000000001, 2.6946924999999999, 2.8663175000000001, 2.9730449999999999, 3.0797724999999998, 3.5164724999999999, 3.8464450000000001, 4.1764175000000003, 4.7008450000000002, 4.8952999999999998, 5.0897550000000003, 5.3429525, 5.4016950000000001, 5.4604375000000003], u'edens': [2.5, 3.1000000000000001, 3.5, 4.9000000000000004]}
{u'pair': [2.5648974999999998, 2.6297950000000001, 2.6946924999999999, 2.8663175000000001, 2.9730449999999999, 3.0797724999999998, 3.5164724999999999, 3.8464450000000001, 4.1764175000000003, 4.7008450000000002, 4.8952999999999998, 5.0897550000000003, 5.3429525, 5.4016950000000001, 5.4604375000000003], u'edens': [2.5, 3.1000000000000001, 3.5, 4.9000000000000004]}


In [9]:
targets['md'] = targets_md['md']
stats['md'] = stats_select_md['md']

In [10]:
# for key in sorted(targets.keys()):
#     for box, xyz, ene in zip(targets[key]['box'], targets[key]['xyz'], stats[key]['energy']):
#         n = len(xyz)
#         vol = np.linalg.det(box)
#         corr = float(n*n)/vol
#         ene.append([corr])

In [11]:
for i, box in enumerate(targets['eos']['box']):
    print(i, box[0,0])

(0, 6.3724472324999999)
(1, 7.0096919557500001)
(2, 7.6469366790000013)
(3, 8.2841814022500007)
(4, 8.9214261255)
(5, 9.5586708487499994)
(6, 10.195915572000001)
(7, 10.833160295250002)
(8, 11.470405018499999)
(9, 12.107649741749999)
(10, 12.744894465)
(11, 13.382139188250001)
(12, 14.0193839115)
(13, 14.656628634749998)
(14, 15.293873357999999)
(15, 15.93111808125)
(16, 16.568362804499998)
(17, 17.205607527750001)
(18, 17.842852251)
(19, 18.480096974249999)
(20, 19.117341697499999)


In [12]:
8.9214261255/12.74489446*2.74

1.9180000007524585

In [13]:
targets['eos_bcc']['box'] = targets['eos_bcc']['box'][0:5]
targets['eos_bcc']['xyz'] = targets['eos_bcc']['xyz'][0:5]
targets['eos_bcc']['temp'] = targets['eos_bcc']['temp'][0:5]
targets['eos_bcc']['beta'] = targets['eos_bcc']['beta'][0:5]
targets['eos_bcc']['energy'] = targets['eos_bcc']['energy'][0:5]
targets['eos_bcc']['forces'] = targets['eos_bcc']['forces'][0:5]
stats['eos_bcc']['energy'] = stats['eos_bcc']['energy'][0:5]
stats['eos_bcc']['forces'] = stats['eos_bcc']['forces'][0:5]

In [14]:
targets['eos']['box'] = targets['eos']['box'][6:9]
targets['eos']['xyz'] = targets['eos']['xyz'][6:9]
targets['eos']['temp'] = targets['eos']['temp'][6:9]
targets['eos']['beta'] = targets['eos']['beta'][6:9]
targets['eos']['energy'] = targets['eos']['energy'][6:9]
targets['eos']['forces'] = targets['eos']['forces'][6:9]
stats['eos']['energy'] = stats['eos']['energy'][6:9]
stats['eos']['forces'] = stats['eos']['forces'][6:9]

In [15]:
targets['relax']['box'][1] = targets['inf']['box'][0]
targets['relax']['xyz'][1] = targets['inf']['xyz'][0]
targets['relax']['temp'][1] = targets['inf']['temp'][0]
targets['relax']['beta'][1] = targets['inf']['beta'][0]
targets['relax']['energy'][1] = targets['inf']['energy'][0]
targets['relax']['forces'][1] = targets['inf']['forces'][0]
stats['relax']['energy'][1] = stats['inf']['energy'][0]
stats['relax']['forces'][1] = stats['inf']['forces'][0]

In [16]:
targets['md']['box'][346:447] = 101*targets['inf']['box']
targets['md']['xyz'][346:447] = 101*targets['inf']['xyz']
targets['md']['temp'][346:447] = 101*targets['inf']['temp']
targets['md']['beta'][346:447] = 101*targets['inf']['beta']
targets['md']['energy'][346:447] = 101*targets['inf']['energy']
targets['md']['forces'][346:447] = 101*targets['inf']['forces']
stats['md']['energy'][346:447] = 101*stats['inf']['energy']
stats['md']['forces'][346:447] = 101*stats['inf']['forces']

In [17]:
targets['relax']['energy']

[-1139.2,
 -2.9688679595213743e-239,
 -1136.4010143821092,
 -1136.7508187321096,
 -1190.2611772847656,
 -1127.1746971578905]

In [18]:
targets['eos_fcc']['weight'] = 0.0
targets['bain']['weight'] = 0.0
targets['eos']['weight'] = 0.0
targets['eos_bcc']['weight'] = 0.0
targets['relax']['weight'] = 0.1
targets['inf']['weight'] = 0.0
#targets['liq_5000K']['weight'] = 0.1

In [19]:
[(key, targets[key]['weight']) for key in targets.keys()]

[('bcc_npt_langevin_3700K', 1.0),
 ('eos', 0.0),
 ('vac_npt_langevin_2000K', 1.0),
 ('md', 1.0),
 ('inf', 0.0),
 ('relax', 0.1),
 ('screw_111_npt_langevin_2000K', 1.0),
 ('bcc_npt_langevin_300K', 1.0),
 ('eos_bcc', 0.0),
 ('liq_5000K', 1.0),
 ('fcc_npt_langevin_300K', 1.0),
 ('vacint', 1.0),
 ('eos_fcc', 0.0),
 ('i110_npt_langevin_2000K', 1.0),
 ('liq_3700K', 1.0),
 ('i111_npt_langevin_2000K', 1.0),
 ('bain', 0.0)]

In [20]:
targets['eos']['temp'] = [5000.0, 5000., 5000., 5000.]
targets['eos']['beta'] = [1/5000.0, 1/5000., 1/5000., 1/5000.]

In [21]:
start_params = []
# start_params.append([ -4.18353755e+00,  -9.63668936e-03,  0.0, -1.00171035e+02,
#          1.90025394e+01,   2.15938317e+01,  -1.39759833e+01,
#          2.16332289e+01,  -3.29542126e+00,   1.70455674e+00,
#          1.41347064e+00,  -9.02958785e-01,  -8.62309098e-01,
#          1.95964588e+00,  -8.70527088e-01,   3.22342700e-02,
#         -1.53866121e+00,   1.37095441e+00,  -4.32896107e+01,
#          4.64461212e+00,   3.23329113e-01,   5.82061842e-02, -0.1])
# start_params.append([ -4.36548916e+00,  -1.11867827e-02,  -1.94093020e+01,
#         -1.40195282e+02,   4.53864231e+01,   1.49756180e+01,
#         -1.36031385e+01,   2.21581200e+01,  -4.61501041e+00,
#          2.06360250e+00,   1.12795645e+00,  -8.01200744e-01,
#         -8.58426028e-01,   1.95860157e+00,  -8.68817546e-01,
#          2.43936831e-02,  -1.54555297e+00,   1.38005891e+00,
#         -5.22754243e+01,   3.98209114e+00,   3.07369702e-01,
#          5.51658454e-02,   1.58601447e+00])
# start_params.append([ -4.16739003e+00,  -4.97036785e-03,  -4.62426912e+02,
#          2.63469650e+02,   1.00253629e+01,   2.05151925e+01,
#         -1.36702712e+01,   2.29129977e+01,  -1.27210473e+01,
#          2.95533783e+00,   8.18470873e-01,  -7.89494754e-01,
#         -8.74455389e-01,   2.10177842e+00,  -9.22682795e-01,
#         -6.93852493e-02,  -1.52166575e+00,   1.40766925e+00,
#          1.52571934e+02,   3.37536873e-01,   6.89310740e-01,
#          4.87102227e-02,   2.79573955e+00])
# start_params.append([ -4.33954858e+00,  -1.02215197e-02,  0.0, -1.12590032e+02,
#          3.06702045e+01,   1.74951979e+01,  -1.33298120e+01,
#          2.13899957e+01,  -3.70881631e+00,   1.56452640e+00,
#          1.27789071e+00,  -8.54623702e-01,  -8.39818015e-01,
#          1.95990543e+00,  -8.75948596e-01,   1.81927625e-02,
#         -1.54683863e+00,   1.38614460e+00,  -4.28747460e+01,
#          4.18974818e+00,   1.56622590e-01,   5.72961538e-02,
#          3.46319317e+00])

# start_params.append([ -4.41712815e+00,  -7.57807965e-03, 0.0, 2.79276814e+02,
#          1.74479561e+02,  -6.59620942e+01,   3.53977481e+01,
#         -1.40546374e+01,   2.11913467e+01,  -4.95603366e+00,
#          8.26392820e-01,   1.26391280e+00,  -9.48799563e-01,
#         -7.79099318e-01,   2.00268261e+00,  -8.90495881e-01,
#         -8.59210993e-03,  -1.54948191e+00,   1.39577855e+00,
#          2.43904610e+02,   3.47694754e+00,  -3.14999583e-01,
#          6.33069119e-02,   5.60287637e+00])

# start_params.append([-4.48347382e+00,  -7.78079006e-03,   2.92055760e+02,
#          1.84809430e+02,  -6.18705307e+01,   3.27735461e+01,
#         -1.36588262e+01,   2.11547368e+01,  -5.45394162e+00,
#          8.02998352e-01,   1.21949599e+00,  -9.43034331e-01,
#         -7.71285316e-01,   2.00564540e+00,  -8.93039729e-01,
#         -9.92574423e-03,  -1.55106037e+00,   1.39898785e+00,
#          2.54912062e+02,   3.19484582e+00,  -3.49947883e-01,
#          6.32970386e-02,   4.13612869e+00])

# start_params.append([ -4.41712815e+00,  -7.57807965e-03, 0.0, 2.79276814e+02,
#          1.74479561e+02,  -6.59620942e+01,   3.53977481e+01,
#         -1.40546374e+01,   2.11913467e+01,  -4.95603366e+00,
#          8.26392820e-01,   1.26391280e+00,  -9.48799563e-01,
#         -7.79099318e-01,   2.00268261e+00,  -8.90495881e-01,
#         -8.59210993e-03,  -1.54948191e+00,   1.39577855e+00,
#          2.43904610e+02,   3.47694754e+00,  -3.14999583e-01,
#          6.33069119e-02])

# start_params.append([ -3.09187534e+00,   1.19622206e-04,  -5.67875917e+02,
#          3.21802084e+02,  -4.71317094e+02,   3.98296604e+02,
#         -1.13270858e+02,   6.03114401e+00,   1.48067908e+01,
#         -1.57531660e+01,   7.80863462e+00,  -2.99246651e+00,
#          7.33596451e-01,  -9.50794219e-01,   1.94186078e+00,
#         -8.83596676e-01,  -1.33608784e-01,  -1.46694544e+00,
#          1.42447434e+00,   7.53865380e+01,   3.29733034e-01,
#          2.18327852e+00,   6.63152141e-02, 0.0])

# start_params.append([ -2.95016477e+00,   3.78177969e-04,  -5.21095611e+02,
#          3.09845244e+02,  -4.69222440e+02,   3.95186037e+02,
#         -1.11630659e+02,   5.36915127e+00,   1.52984702e+01,
#         -1.54915170e+01,   7.53107481e+00,  -2.98591742e+00,
#          7.55346307e-01,  -1.00678488e+00,   1.96634582e+00,
#         -8.71560927e-01,  -1.41820832e-01,  -1.46508000e+00,
#          1.42936365e+00,   7.23805795e+01,   0.0, 9.18071460e-01,
#          2.04259992e+00,   6.28575567e-02,   7.62804667e-03])

# start_params.append([ -3.27882058e+00,  -9.77681860e-04,  -5.95486537e+02,
#          3.42401535e+02,  -4.15497479e+02,   3.71316746e+02,
#         -1.07274610e+02,  -2.93282898e+00,   1.51298572e+01,
#         -1.37454674e+01,   8.16724633e+00,  -3.16850352e+00,
#          8.74607042e-01,  -1.07878768e+00,   1.97939820e+00,
#         -8.26149646e-01,  -1.62437297e-01,  -1.45694910e+00,
#          1.42153527e+00,   1.29203135e+02,  -9.25284676e+00,
#          2.63967417e+00,   2.06410900e+00,   6.03258240e-02*2])

# start_params.append([ -3.56709315e+00,  -2.97358686e-03,  -4.37632666e+02,
#          3.35095381e+02,  -2.95083648e+02,   2.94263473e+02,
#         -8.57731610e+01,  -1.62036228e+01,   1.30838141e+01,
#         -6.46551376e+00,   8.01826587e+00,  -3.58786095e+00,
#          1.25735987e+00,  -1.21627097e+00,   2.04033545e+00,
#         -8.04702913e-01,  -1.92666480e-01,  -1.44083371e+00,
#          1.41816098e+00,   1.70360405e+02,  -1.89070431e+01,
#          7.47139804e+00,   1.20540194e+00,   1.34588181e-01, 0.0])

# start_params.append([ -3.89054208e+00,  -2.99469508e-03,  -7.60325091e+02,
#          2.25922421e+02,   4.31682139e+01,  -2.00879587e+01,
#          3.15977299e+01,  -1.41216383e+01,   1.40729549e+01,
#         -2.29741069e-01,   5.66020784e-01,   9.50813370e-01,
#         -7.07877506e-01,  -6.33885976e-01,   2.11514130e+00,
#         -1.07084535e+00,  -1.99471358e-01,  -1.53976066e+00,
#          1.54749579e+00,   1.94668273e+02,  -1.93218649e+00,
#          4.87330211e+00,  -4.95019886e-01,   1.54561460e-01])

# start_params.append([ -3.01252922e+00,   3.34774026e-03,  -3.63245948e+02,
#          7.08473344e+01,   5.18269140e+00,  -4.21852665e+01,
#          5.03986439e+01,  -3.49299323e+01,   9.39050841e+00,
#          7.10916965e+00,   2.53141800e+00,  -1.27714028e-01,
#         -8.13152443e-01,  -5.45294030e-01,   2.19662624e+00,
#         -1.03021233e+00,  -2.37376783e-01,  -1.54945600e+00,
#          1.51410989e+00,   8.93873263e+01,  -2.44415218e+01,
#          1.44237601e+01,  -2.48517486e-01,   1.57735701e-01,
#         0.0])

# start_params.append([ -2.76762846e+00,   1.89330096e-03,  -3.78969507e+02,
#          4.97375946e+01,   5.61080362e+01,  -8.13991417e+01,
#          6.18213043e+01,  -3.36881958e+01,   9.64679581e+00,
#          6.37021438e+00,   2.56205745e+00,  -6.89271288e-02,
#         -8.28256696e-01,  -5.90663693e-01,   2.19738636e+00,
#         -9.78519868e-01,  -2.34054246e-01,  -1.56404102e+00,
#          1.50934562e+00,   1.25199350e+02,  -2.58933860e+01,
#          1.60206690e+01,  -2.46344691e-01,   1.98323959e-01,
#         0.0])

# start_params.append([ -2.96336623e+00,   9.35841772e-04,  -4.69405427e+02,
#          6.56489326e+01,   6.59098372e+01,  -6.95558862e+01,
#          5.05321941e+01,  -2.68901225e+01,   6.12983979e+00,
#          8.40257564e+00,   2.03599185e+00,   1.44655457e-01,
#         -8.07626370e-01,  -6.77900254e-01,   2.19493850e+00,
#         -9.02854774e-01,  -2.35346587e-01,  -1.56947636e+00,
#          1.49891405e+00,   1.54692423e+02,  -1.84128239e+01,
#          1.40416342e+01,  -4.74336808e-01,   2.07514499e-01])

# start_params.append([ -2.83157185e+00,   8.39115358e-04,  -4.21330968e+02,
#          5.62354559e+01,   6.55868693e+01,  -7.39097694e+01,
#          5.32753168e+01,  -2.90768080e+01,   9.73480693e+00,
#          5.86938247e+00,   2.40344261e+00,   1.33807130e-01,
#         -7.34188649e-01,  -8.53691403e-01,   2.20105517e+00,
#         -7.61586871e-01,  -2.42532670e-01,  -1.58457873e+00,
#          1.47655687e+00,   0.0, 1.48755630e+02,  0.0, -1.98109911e+01,
#          1.44803287e+01,  -1.17489342e-01,   2.06651900e-01])

# start_params.append([ -3.91274686e+00,  -5.02814282e-03,  -3.60607144e+02,
#          3.08180837e+02,  -2.45731624e+02,   2.57711860e+02,
#         -7.57560376e+01,  -3.26396342e+01,   1.25979333e+01,
#         -3.17982854e+00,   7.88108320e+00,  -3.46868286e+00,
#          1.29424331e+00,  -1.31629778e+00,   2.05422384e+00,
#         -8.33747039e-01,  -1.75108086e-01,  -1.43404985e+00,
#          1.45403961e+00,   1.43868098e+02,  -2.72551797e+01,
#          8.05592823e+00,   1.03649030e+00,   1.04857599e-01,
#         -4.20237149e+01])

# start_params.append([ -3.34519702e+00,   7.66822271e-04,  -4.03553412e+02,
#          1.55282185e+01,   2.90516401e+01,  -6.23087513e+01,
#          8.48184009e+01, -3.48806112e+01,   9.13550274e+00,
#          5.96096071e+00,   2.54062374e+00,   2.39873080e-01,
#         -7.19239652e-01,  -1.04267291e+00,   2.29366373e+00,
#         -7.05206149e-01,  -2.72279799e-01,  -1.55954882e+00,
#          1.45964251e+00,  -2.01431222e+02,   3.24297105e+01,
#          3.81664135e+01,  -2.18948938e+01,   1.02484895e+01,
#          2.20300623e-01,   1.13874983e-01])

# start_params.append([ -4.65804603e+00,  -7.56457588e-03,  -1.68777520e+02,
#          1.56052349e+02,   1.42649908e+01,  -4.55941009e+00,
#          1.97613340e+01,   4.84204936e+01,   6.49466770e+00,
#         -6.62233324e+00,  -2.16547494e+00,   1.30932417e+00,
#         -8.33281480e-01,  -8.32333597e-01,   2.09417158e+00,
#         -6.19566076e-01,  -9.53112547e-02,  -1.49756931e+00,
#          1.27167940e+00,   0.0, 1.06853821e+02,   2.74234479e+01,
#         -1.26300179e+00,  -1.66378666e+00,   2.24848394e-01])

# start_params.append([
#         -4.20657287, -0.00479083415,
#         -4.25919648e+03,   1.39193989e+02,   2.95719230e+01,
#         -1.18147968e+01,   1.70731105e+01,   4.70939187e+01,
#          8.92934163e+00,  -8.86791838e+00,  -1.76587802e+00,
#          1.76893096e+00,  -1.34741230e+00,  -4.32516046e-01,
#          1.42269060e+00,  -4.94492898e-01,  -6.03661025e-02,
#         -5.84036559e-01,   5.77793165e-01,
#         -3037.16289, 125.775067, 32.7634816,
#         -1.80274854, -1.77552231, 0.210521327
# ])

# start_params.append([-4.22162443, -0.00407859333,
#                      3478.84405,113.46408,-0.00903949895,1.35993297,
#                      12.8147257,46.4943117,6.83202558,-7.30628605,
#                      -1.99295103,1.90244807,-1.31365252,-0.804929717,
#                      1.98644116,-0.488771708,-0.0443359448,-1.46775185,
#                      1.19532238,-0.00538433431,-0.00128544755,-0.0025275236,
#                      2965.39266, 95.3494422, 31.2833793, -1.50835193, -1.76957075,0.205860236
#                     ]
# )

# start_params.append([ -4.19976555e+00,  -3.91095024e-03,   4.11762114e+03,
#          1.06506552e+02,   5.29253551e-01,   1.24324874e+00,
#          1.19653127e+01,   4.68002406e+01,   6.67699247e+00,
#         -7.04927872e+00,  -2.05647856e+00,   1.95042373e+00,
#         -1.33520940e+00,  -8.11118459e-01,   1.99354705e+00,
#         -4.84402488e-01,  -4.71892464e-02,  -1.46523753e+00,
#          1.19717936e+00,  -6.80779121e-03,  -3.49337274e-03,
#         -3.76805537e-03,   3.51929579e+03,   9.27623682e+01,
#          3.15808969e+01,  -1.44019460e+00,  -1.78665037e+00,
#          2.05532639e-01])

# start_params.append([ -4.16353110e+00,  -3.64115525e-03,   0.0, 4.52561445e+03,
#          1.07488634e+02,  -1.52604631e+01,   8.88626493e+00,
#          1.07120352e+01,   4.61310765e+01,   6.70641338e+00,
#         -7.14000617e+00,  -2.02174483e+00,   1.98976760e+00,
#         -1.35491449e+00,  -8.12592118e-01,   1.99278451e+00,
#         -4.84145757e-01,  -4.78673818e-02,  -1.46201396e+00,
#          1.19810233e+00,  -7.36954413e-03,  -3.88205423e-03,
#         -5.12566656e-03,   3.98736133e+03,   8.58053818e+01,
#          3.17983469e+01,  -1.52861585e+00,  -1.77508182e+00,
#          2.07405945e-01])

# start_params.append([ -4.08460829e+00,  -1.38588599e-03,  -1.32507730e+04,
#          1.93928425e+03,   2.05970852e+01,  -2.37669887e+02,
#          1.21943656e+02,  -3.24296128e+01,   6.26409270e+01,
#          8.36697799e+00,  -9.54708966e+00,  -2.06899378e+00,
#          1.75611005e+00,  -1.35952781e+00,  -7.35465370e-01,
#          1.98598708e+00,  -4.73450090e-01,  -4.99940237e-02,
#         -1.45135718e+00,   1.19557147e+00,   1.75773867e-04,
#         -1.84569288e-02,  -2.63221509e-02,   1.65754590e+03,
#         -1.03315521e+02,   3.86317998e+01,  -1.37580192e+00,
#         -2.15109594e+00,   2.00163255e-01])

# start_params.append([-5.946454, -0.049477, 
#                      9.60851701e+01, -1.84410924e+02,   9.35784080e+01,  -7.98358265e+00,
#                    7.47034093e+00,  -1.52756044e+00,   1.25205933e+00,
#                    1.63082162e+00,  -1.41854775e+00,  -8.19936046e-01,
#                    1.98013514e+00,  -6.96430180e-01,   3.04546910e-02,
#                   -1.63131143e+00,   1.38409896e+00,
#                      -4.20429108,  0.5182177 ,  0.05627208,  0.03441642])

# start_params.append([ -5.94255002e+00,  -4.52753307e-03,  -8.98973159e+01,
#          1.58307190e+02,  -7.12304117e+01,   6.46096124e+00,
#          3.42388839e-01,   6.02193303e+00,   1.71026053e+00,
#          8.61616548e-01,  -1.24021132e+00,  -7.52778245e-01,
#          2.16478392e+00,  -6.80500465e-01,   8.35831474e-03,
#         -1.70858135e+00,   1.33828368e+00,   1.06188317e+01,
#          2.26603603e+00,  -1.52122221e-02,   2.00566458e-02])

start_params.append([ -6.09401647e+00,  -1.82769062e-03,
                     
                     -7.89450478e+01,
         1.42266112e+02,  -6.39859044e+01,   8.66934800e+00,
         1.56538761e+00,  -2.26299556e+00,   3.99827950e+00,
         8.78041445e-01,  -1.38562030e+00,  -8.08118622e-01,
         2.25072617e+00,  -6.88281869e-01,  -1.89479049e-02,
        -1.70456724e+00,   1.33270394e+00,
                     
                    9.82708723e+00, 7.44969065e-01,   4.95175701e-01,   7.69627079e-03])

multi_pars = start_params
print(multi_pars)

[[-6.09401647, -0.00182769062, -78.9450478, 142.266112, -63.9859044, 8.669348, 1.56538761, -2.26299556, 3.9982795, 0.878041445, -1.3856203, -0.808118622, 2.25072617, -0.688281869, -0.0189479049, -1.70456724, 1.33270394, 9.82708723, 0.744969065, 0.495175701, 0.00769627079]]


In [ ]:
def optimize_EAM_mp(pars, targets, stats, utot_func):
    
    output = fmin(sd2_loss, pars, args=(targets, stats, utot_EAM_per_atom, None, 0.05, 1), maxiter=100000, maxfun=100000, disp=0, full_output=1,ftol=1e-6)

    return tuple([output[1], output[0]])

In [ ]:
params_output = []

pool = mp.Pool()

for ieam in range(1):#5, len(stats_opts[it]['hyperparams'])):

    get_sd = functools.partial(sd2_loss, targets=targets, stats=stats, utot_func=utot_EAM_per_atom, ftot_func=None, dl=0.05, verbose=1)

    get_opt = functools.partial(optimize_EAM_mp, targets=targets, stats=stats, utot_func=utot_EAM_per_atom)

    # initial ordering
    print('Initial params:', multi_pars[-1])
    
    #distances = pool.map(get_sd, multi_pars)
    
    distances = [get_sd(multi_pars[-1])]
    
    optimal_params = zip(distances, multi_pars[-1:])
    best_params = sorted(optimal_params, key=lambda param: param[0], reverse=True)
    m_pars = [p[1] for p in best_params]

    print('Best params:', best_params)
    print('ieam:', ieam)
    #m_pars = multi_pars

    for i in range(50):
        #optimal_params = pool.map(get_opt, m_pars)
        optimal_params = [get_opt(m_pars)]
        best_params = sorted(optimal_params, key=lambda param: param[0], reverse=True)
        m_pars = [p[1] for p in best_params]
        
        print("Iteration {}, best params: {}".format(i, best_params))

    print('Final Best params:', best_params)
    
    params_output.append(best_params)

pool.close()
pool.join()

('Initial params:', [-6.09401647, -0.00182769062, -78.9450478, 142.266112, -63.9859044, 8.669348, 1.56538761, -2.26299556, 3.9982795, 0.878041445, -1.3856203, -0.808118622, 2.25072617, -0.688281869, -0.0189479049, -1.70456724, 1.33270394, 9.82708723, 0.744969065, 0.495175701, 0.00769627079])
loss 2.83055310243e-06 2.83055310243e-06 0.0
('Best params:', [(2.8305531024263108e-06, [-6.09401647, -0.00182769062, -78.9450478, 142.266112, -63.9859044, 8.669348, 1.56538761, -2.26299556, 3.9982795, 0.878041445, -1.3856203, -0.808118622, 2.25072617, -0.688281869, -0.0189479049, -1.70456724, 1.33270394, 9.82708723, 0.744969065, 0.495175701, 0.00769627079])])
('ieam:', 0)
loss 2.83055310243e-06 2.83055310243e-06 0.0
loss 0.00700558626733 0.00700558626733 0.0
loss 2.84423613253e-06 2.84423613253e-06 0.0
loss 3.28792679212e-06 3.28792679212e-06 0.0
loss 9.08916245246e-06 9.08916245246e-06 0.0
loss 7.53760200476e-06 7.53760200476e-06 0.0
loss 4.40095765814e-06 4.40095765814e-06 0.0
loss 3.25020806224

loss 2.832006404e-06 2.832006404e-06 0.0
loss 2.83524945069e-06 2.83524945069e-06 0.0
loss 2.83580722033e-06 2.83580722033e-06 0.0
loss 2.83197817011e-06 2.83197817011e-06 0.0
loss 2.83510499442e-06 2.83510499442e-06 0.0
loss 2.83526572056e-06 2.83526572056e-06 0.0
loss 2.83191224342e-06 2.83191224342e-06 0.0
loss 2.83490663624e-06 2.83490663624e-06 0.0
loss 2.83743934018e-06 2.83743934018e-06 0.0
loss 2.83163379987e-06 2.83163379987e-06 0.0
loss 2.83492592471e-06 2.83492592471e-06 0.0
loss 2.83186911066e-06 2.83186911066e-06 0.0
loss 2.83597039665e-06 2.83597039665e-06 0.0
loss 2.8316564635e-06 2.8316564635e-06 0.0
loss 2.83674175601e-06 2.83674175601e-06 0.0
loss 2.83157764576e-06 2.83157764576e-06 0.0
loss 2.83654586139e-06 2.83654586139e-06 0.0
loss 2.83157148836e-06 2.83157148836e-06 0.0
loss 2.83765732583e-06 2.83765732583e-06 0.0
loss 2.8314469878e-06 2.8314469878e-06 0.0
loss 2.83499583148e-06 2.83499583148e-06 0.0
loss 2.8316857719e-06 2.8316857719e-06 0.0
loss 2.83765540478e-

loss 2.83061126167e-06 2.83061126167e-06 0.0
loss 2.83055164041e-06 2.83055164041e-06 0.0
loss 2.83061811992e-06 2.83061811992e-06 0.0
loss 2.83054992368e-06 2.83054992368e-06 0.0
loss 2.83059644023e-06 2.83059644023e-06 0.0
loss 2.8305515249e-06 2.8305515249e-06 0.0
loss 2.83057448983e-06 2.83057448983e-06 0.0
loss 2.83057705319e-06 2.83057705319e-06 0.0
loss 2.83058183637e-06 2.83058183637e-06 0.0
loss 2.83054718793e-06 2.83054718793e-06 0.0
loss 2.83057966083e-06 2.83057966083e-06 0.0
loss 2.83054722007e-06 2.83054722007e-06 0.0
loss 2.83056663479e-06 2.83056663479e-06 0.0
loss 2.83058182973e-06 2.83058182973e-06 0.0
loss 2.83054573617e-06 2.83054573617e-06 0.0
loss 2.83057201755e-06 2.83057201755e-06 0.0
loss 2.83054583178e-06 2.83054583178e-06 0.0
loss 2.83058778627e-06 2.83058778627e-06 0.0
loss 2.83054281345e-06 2.83054281345e-06 0.0
loss 2.83057878255e-06 2.83057878255e-06 0.0
loss 2.83054365206e-06 2.83054365206e-06 0.0
loss 2.83057339292e-06 2.83057339292e-06 0.0
loss 2.83054

loss 2.83048868969e-06 2.83048868969e-06 0.0
loss 2.83049102067e-06 2.83049102067e-06 0.0
loss 2.83048649137e-06 2.83048649137e-06 0.0
loss 2.83048827538e-06 2.83048827538e-06 0.0
loss 2.83048860283e-06 2.83048860283e-06 0.0
loss 2.83049417744e-06 2.83049417744e-06 0.0
loss 2.83048555767e-06 2.83048555767e-06 0.0
loss 2.83049174871e-06 2.83049174871e-06 0.0
loss 2.83048575758e-06 2.83048575758e-06 0.0
loss 2.8304891525e-06 2.8304891525e-06 0.0
loss 2.8304900706e-06 2.8304900706e-06 0.0
loss 2.83048588337e-06 2.83048588337e-06 0.0
loss 2.83048920577e-06 2.83048920577e-06 0.0
loss 2.83048596163e-06 2.83048596163e-06 0.0
loss 2.83048755252e-06 2.83048755252e-06 0.0
loss 2.83048684164e-06 2.83048684164e-06 0.0
loss 2.83049064996e-06 2.83049064996e-06 0.0
loss 2.83048555129e-06 2.83048555129e-06 0.0
loss 2.83048737644e-06 2.83048737644e-06 0.0
loss 2.83049134573e-06 2.83049134573e-06 0.0
loss 2.83048537356e-06 2.83048537356e-06 0.0
loss 2.83049034095e-06 2.83049034095e-06 0.0
loss 2.8304854

loss 2.83046974103e-06 2.83046974103e-06 0.0
loss 2.83046903142e-06 2.83046903142e-06 0.0
loss 2.83047015447e-06 2.83047015447e-06 0.0
loss 2.83046961929e-06 2.83046961929e-06 0.0
loss 2.83046939933e-06 2.83046939933e-06 0.0
loss 2.83046813296e-06 2.83046813296e-06 0.0
loss 2.83046668377e-06 2.83046668377e-06 0.0
loss 2.83046711479e-06 2.83046711479e-06 0.0
loss 2.83046838051e-06 2.83046838051e-06 0.0
loss 2.83046736135e-06 2.83046736135e-06 0.0
loss 2.83046673744e-06 2.83046673744e-06 0.0
loss 2.83046606103e-06 2.83046606103e-06 0.0
loss 2.83046518674e-06 2.83046518674e-06 0.0
loss 2.83046608638e-06 2.83046608638e-06 0.0
loss 2.83046594662e-06 2.83046594662e-06 0.0
loss 2.83046578262e-06 2.83046578262e-06 0.0
loss 2.83046436086e-06 2.83046436086e-06 0.0
loss 2.83046279114e-06 2.83046279114e-06 0.0
loss 2.83046283153e-06 2.83046283153e-06 0.0
loss 2.83046405819e-06 2.83046405819e-06 0.0
loss 2.83046342156e-06 2.83046342156e-06 0.0
loss 2.83046388074e-06 2.83046388074e-06 0.0
loss 2.830

...

loss 2.82074894216e-06 2.82074894216e-06 0.0
loss 2.82074882811e-06 2.82074882811e-06 0.0
loss 2.82074878772e-06 2.82074878772e-06 0.0
loss 2.82074882455e-06 2.82074882455e-06 0.0
loss 2.82074882298e-06 2.82074882298e-06 0.0
loss 2.82074876663e-06 2.82074876663e-06 0.0
loss 2.82074880042e-06 2.82074880042e-06 0.0
loss 2.82074874182e-06 2.82074874182e-06 0.0
loss 2.82074867349e-06 2.82074867349e-06 0.0
loss 2.82074869981e-06 2.82074869981e-06 0.0
loss 2.82074872127e-06 2.82074872127e-06 0.0
loss 2.82074879232e-06 2.82074879232e-06 0.0
loss 2.82074869751e-06 2.82074869751e-06 0.0
loss 2.82074870722e-06 2.82074870722e-06 0.0
loss 2.82074866349e-06 2.82074866349e-06 0.0
loss 2.82074857301e-06 2.82074857301e-06 0.0
loss 2.82074864579e-06 2.82074864579e-06 0.0
loss 2.82074870114e-06 2.82074870114e-06 0.0
loss 2.82074864805e-06 2.82074864805e-06 0.0
loss 2.82074862655e-06 2.82074862655e-06 0.0
loss 2.82074866317e-06 2.82074866317e-06 0.0
loss 2.82074861777e-06 2.82074861777e-06 0.0
loss 2.820

loss 2.82071571634e-06 2.82071571634e-06 0.0
loss 2.82071532366e-06 2.82071532366e-06 0.0
loss 2.82071357162e-06 2.82071357162e-06 0.0
loss 2.82071439745e-06 2.82071439745e-06 0.0
loss 2.82071263926e-06 2.82071263926e-06 0.0
loss 2.82071286394e-06 2.82071286394e-06 0.0
loss 2.82071295062e-06 2.82071295062e-06 0.0
loss 2.8207118536e-06 2.8207118536e-06 0.0
loss 2.82071056859e-06 2.82071056859e-06 0.0
loss 2.82071354229e-06 2.82071354229e-06 0.0
loss 2.82071333536e-06 2.82071333536e-06 0.0
loss 2.82071199873e-06 2.82071199873e-06 0.0
loss 2.82071157063e-06 2.82071157063e-06 0.0
loss 2.82071126853e-06 2.82071126853e-06 0.0
loss 2.82071092525e-06 2.82071092525e-06 0.0
loss 2.82071121105e-06 2.82071121105e-06 0.0
loss 2.82071079489e-06 2.82071079489e-06 0.0
loss 2.82071124711e-06 2.82071124711e-06 0.0
loss 2.82071019573e-06 2.82071019573e-06 0.0
loss 2.82070998181e-06 2.82070998181e-06 0.0
loss 2.82071177905e-06 2.82071177905e-06 0.0
loss 2.82071133076e-06 2.82071133076e-06 0.0
loss 2.82070

loss 2.82068879823e-06 2.82068879823e-06 0.0
loss 2.8206880004e-06 2.8206880004e-06 0.0
loss 2.8206896226e-06 2.8206896226e-06 0.0
loss 2.820687354e-06 2.820687354e-06 0.0
loss 2.82068924062e-06 2.82068924062e-06 0.0
loss 2.82068734467e-06 2.82068734467e-06 0.0
loss 2.82068654037e-06 2.82068654037e-06 0.0
loss 2.82068891731e-06 2.82068891731e-06 0.0
loss 2.82068728407e-06 2.82068728407e-06 0.0
loss 2.82068579944e-06 2.82068579944e-06 0.0
loss 2.82068556365e-06 2.82068556365e-06 0.0
loss 2.82068825387e-06 2.82068825387e-06 0.0
loss 2.8206879027e-06 2.8206879027e-06 0.0
loss 2.82068836894e-06 2.82068836894e-06 0.0
loss 2.82068706166e-06 2.82068706166e-06 0.0
loss 2.82068568913e-06 2.82068568913e-06 0.0
loss 2.82068604393e-06 2.82068604393e-06 0.0
loss 2.82068916117e-06 2.82068916117e-06 0.0
loss 2.82068676597e-06 2.82068676597e-06 0.0
loss 2.82068778585e-06 2.82068778585e-06 0.0
loss 2.82068675541e-06 2.82068675541e-06 0.0
loss 2.82068779402e-06 2.82068779402e-06 0.0
loss 2.82068607633e-

loss 2.82067939817e-06 2.82067939817e-06 0.0
loss 2.8206796832e-06 2.8206796832e-06 0.0
loss 2.82068008909e-06 2.82068008909e-06 0.0
loss 2.82067944026e-06 2.82067944026e-06 0.0
loss 2.82067991374e-06 2.82067991374e-06 0.0
loss 2.82068015568e-06 2.82068015568e-06 0.0
loss 2.82067939849e-06 2.82067939849e-06 0.0
loss 2.82067951292e-06 2.82067951292e-06 0.0
loss 2.82067993651e-06 2.82067993651e-06 0.0
loss 2.82067937936e-06 2.82067937936e-06 0.0
loss 2.82067999517e-06 2.82067999517e-06 0.0
loss 2.82067936769e-06 2.82067936769e-06 0.0
loss 2.82067970361e-06 2.82067970361e-06 0.0
loss 2.82067976466e-06 2.82067976466e-06 0.0
loss 2.82067965938e-06 2.82067965938e-06 0.0
loss 2.8206791335e-06 2.8206791335e-06 0.0
loss 2.82067979404e-06 2.82067979404e-06 0.0
loss 2.820679818e-06 2.820679818e-06 0.0
loss 2.82067961059e-06 2.82067961059e-06 0.0
loss 2.82067931927e-06 2.82067931927e-06 0.0
loss 2.82067990125e-06 2.82067990125e-06 0.0
loss 2.82067927344e-06 2.82067927344e-06 0.0
loss 2.82067945902

loss 2.8206789376e-06 2.8206789376e-06 0.0
loss 2.82067894525e-06 2.82067894525e-06 0.0
loss 2.82067895946e-06 2.82067895946e-06 0.0
loss 2.82067893651e-06 2.82067893651e-06 0.0
loss 2.82067896754e-06 2.82067896754e-06 0.0
loss 2.82067893696e-06 2.82067893696e-06 0.0
loss 2.82067895792e-06 2.82067895792e-06 0.0
loss 2.82067893624e-06 2.82067893624e-06 0.0
loss 2.82067894596e-06 2.82067894596e-06 0.0
loss 2.82067895077e-06 2.82067895077e-06 0.0
loss 2.82067893687e-06 2.82067893687e-06 0.0
loss 2.82067895142e-06 2.82067895142e-06 0.0
loss 2.82067893628e-06 2.82067893628e-06 0.0
loss 2.82067893856e-06 2.82067893856e-06 0.0
loss 2.8206789536e-06 2.8206789536e-06 0.0
loss 2.82067893568e-06 2.82067893568e-06 0.0
loss 2.82067895208e-06 2.82067895208e-06 0.0
loss 2.82067893656e-06 2.82067893656e-06 0.0
loss 2.82067894961e-06 2.82067894961e-06 0.0
loss 2.8206789364e-06 2.8206789364e-06 0.0
loss 2.82067893989e-06 2.82067893989e-06 0.0
loss 2.82067895385e-06 2.82067895385e-06 0.0
loss 2.820678934

In [ ]:
[ -5.96626153e+00,  -5.17518556e-03,  -4.96983693e+01,
         1.22321937e+02,  -5.35162766e+01,   1.58488081e+00,
         7.75828614e+00,  -1.36165446e+00,   2.92588751e+00,
         7.77344387e-01,  -1.28829453e+00,  -7.73687596e-01,
         2.19815183e+00,  -6.81125895e-01,   5.29527859e-04,
        -1.70503307e+00,   1.33193410e+00,   1.84927281e+01,
         1.51379529e+00,   1.84818095e-01,   1.62793488e-02]


# params_output = [ -4.72546533e+00,  -6.97068121e-03,  -3.51290250e+02,
#          1.63175272e+02,   9.43598046e+00,  -3.25684355e+00,
#          2.30446047e+01,   4.06678582e+01,   7.56509137e+00,
#         -7.51663487e+00,  -1.59146529e+00,   1.14098894e+00,
#         -8.25274736e-01,  -8.09954101e-01,   2.09984210e+00,
#         -6.52067950e-01,  -8.56530962e-02,  -1.49919134e+00,
#          1.27499075e+00,   1.07226915e+02,   2.36918896e+01,
#         -1.29592031e+00,  -1.49967198e+00,   2.14664790e-01]
# params_output = [[ -4.65804603e+00,  -7.56457588e-03,  -1.68777520e+02,
#          1.56052349e+02,   1.42649908e+01,  -4.55941009e+00,
#          1.97613340e+01,   4.84204936e+01,   6.49466770e+00,
#         -6.62233324e+00,  -2.16547494e+00,   1.30932417e+00,
#         -8.33281480e-01,  -8.32333597e-01,   2.09417158e+00,
#         -6.19566076e-01,  -9.53112547e-02,  -1.49756931e+00,
#          1.27167940e+00,   1.06853821e+02,   2.74234479e+01,
#         -1.26300179e+00,  -1.66378666e+00,   2.24848394e-01]]

# params_output = [[ -4.20657287e+00,  -4.79083415e-03,  -4.28662319e+03,
#          1.39452507e+02,   2.91473879e+01,  -1.08844243e+01,
#          1.59102238e+01,   4.86110415e+01,   7.30591870e+00,
#         -8.24225202e+00,  -1.76541035e+00,   1.69386336e+00,
#         -1.23281961e+00,  -7.91007042e-01,   1.98078659e+00,
#         -4.92223582e-01,  -4.64721217e-02,  -1.47436944e+00,
#          1.19244103e+00,  -3.03716289e+03,   1.25775067e+02,
#          3.27634816e+01,  -1.80274854e+00,  -1.77552231e+00,
#          2.10521327e-01]]

# params_output = [[ -4.18519141e+00,  -3.73537266e-03,   3.96171752e+03,
#          1.18316128e+02,  -8.46735170e+00,   1.49884675e+01,
#          5.43882551e+00,   4.19584851e+01,   1.05383775e+01,
#         -8.44501744e+00,  -1.76769900e+00,   2.25806651e+00,
#         -1.68693854e+00,  -4.53202950e-01,   1.47888419e+00,
#         -4.72027075e-01,  -7.36382597e-02,  -5.90831325e-01,
#          5.77070122e-01,   3.49058280e+03,   9.94638189e+01,
#          2.96556203e+01,  -7.14356623e-01,  -1.77478736e+00,
#          2.05018897e-01]] # s2 = 1.5313539642322428e-06

# params_output = [[ -4.19976555e+00,  -3.91095024e-03,   4.11762114e+03,
#          1.06506552e+02,   5.29253551e-01,   1.24324874e+00,
#          1.19653127e+01,   4.68002406e+01,   6.67699247e+00,
#         -7.04927872e+00,  -2.05647856e+00,   1.95042373e+00,
#         -1.33520940e+00,  -8.11118459e-01,   1.99354705e+00,
#         -4.84402488e-01,  -4.71892464e-02,  -1.46523753e+00,
#          1.19717936e+00,  -6.80779121e-03,  -3.49337274e-03,
#         -3.76805537e-03,   3.51929579e+03,   9.27623682e+01,
#          3.15808969e+01,  -1.44019460e+00,  -1.78665037e+00,
#          2.05532639e-01]]

# params_output = [[ -4.16353110e+00,  -3.64115525e-03,   4.52561445e+03,
#          1.07488634e+02,  -1.52604631e+01,   8.88626493e+00,
#          1.07120352e+01,   4.61310765e+01,   6.70641338e+00,
#         -7.14000617e+00,  -2.02174483e+00,   1.98976760e+00,
#         -1.35491449e+00,  -8.12592118e-01,   1.99278451e+00,
#         -4.84145757e-01,  -4.78673818e-02,  -1.46201396e+00,
#          1.19810233e+00,  -7.36954413e-03,  -3.88205423e-03,
#         -5.12566656e-03,   3.98736133e+03,   8.58053818e+01,
#          3.17983469e+01,  -1.52861585e+00,  -1.77508182e+00,
#          2.07405945e-01]]

# params_output = [[ -4.08460829e+00,  -1.38588599e-03,  -1.32507730e+04,
#          1.93928425e+03,   2.05970852e+01,  -2.37669887e+02,
#          1.21943656e+02,  -3.24296128e+01,   6.26409270e+01,
#          8.36697799e+00,  -9.54708966e+00,  -2.06899378e+00,
#          1.75611005e+00,  -1.35952781e+00,  -7.35465370e-01,
#          1.98598708e+00,  -4.73450090e-01,  -4.99940237e-02,
#         -1.45135718e+00,   1.19557147e+00,   1.75773867e-04,
#         -1.84569288e-02,  -2.63221509e-02,   1.65754590e+03,
#         -1.03315521e+02,   3.86317998e+01,  -1.37580192e+00,
#         -2.15109594e+00,   2.00163255e-01]]

params_output = [[ -5.94255002e+00,  -4.52753307e-03,  -8.98973159e+01,
         1.58307190e+02,  -7.12304117e+01,   6.46096124e+00,
         3.42388839e-01,   6.02193303e+00,   1.71026053e+00,
         8.61616548e-01,  -1.24021132e+00,  -7.52778245e-01,
         2.16478392e+00,  -6.80500465e-01,   8.35831474e-03,
        -1.70858135e+00,   1.33828368e+00,   1.06188317e+01,
         2.26603603e+00,  -1.52122221e-02,   2.00566458e-02]]

params_output.append([ -6.67485922e+00,  -1.02376651e-02,   1.91540112e+02,
        -1.02318453e+02,   3.55386850e+01,  -5.52132877e+00,
         9.33351478e+00,   9.88401954e-01,   1.50327402e+00,
        -5.01846513e-01,  -1.06459763e+00,  -5.05974054e-01,
         2.00937835e+00,  -6.02252894e-01,   8.10245260e-04,
        -1.64969075e+00,   1.29187809e+00,   4.66926359e+01,
         2.30114335e+00,  -7.42024973e-01,   4.07443544e-02])

params_output.append([ -6.09401647e+00,  -1.82769062e-03,  -7.89450478e+01,
         1.42266112e+02,  -6.39859044e+01,   8.66934800e+00,
         1.56538761e+00,  -2.26299556e+00,   3.99827950e+00,
         8.78041445e-01,  -1.38562030e+00,  -8.08118622e-01,
         2.25072617e+00,  -6.88281869e-01,  -1.89479049e-02,
        -1.70456724e+00,   1.33270394e+00,   9.82708723e+00,
         7.44969065e-01,   4.95175701e-01,   7.69627079e-03])

print(params_output)

In [ ]:
# convert lists of optimal parameters to dictionaries
params_dicts = []
for params in params_output:
    params_dicts.append(to_param_dict(params, stats['hyperparams']))

In [ ]:
params_dicts

In [ ]:
file_name = os.path.join(working, "params_out.pickle")

for params in params_dicts:
    params_to_pickle(params, file_name)

In [ ]:
with open(os.path.join(working, 'params_out' + '.pickle'), 'rb') as fi:
    retreived_params = pickle.load(fi)  

In [ ]:
retreived_params

In [ ]:
retreived_params[-2:]